In [224]:
using System.IO;
using System.Text.RegularExpressions;
using Microsoft.DotNet.Interactive.Formatting;
using System.Globalization;
using System.Text.Json;
using Microsoft.DotNet.Interactive.Formatting.TabularData;
static class RegexHelper
{
    public static T As<T>(Regex regex, string s, IFormatProvider? provider = null) where T: struct
    {
        var match = regex.Match(s);
        if (!match.Success) throw new InvalidOperationException($"input '{s}' does not match regex '{regex}'");

        var constructor = typeof(T).GetConstructors().Single();
        
        var j = from p in constructor.GetParameters()
                join m in match.Groups.OfType<Group>() on p.Name equals m.Name
                select Convert.ChangeType(m.Value, p.ParameterType, provider ?? CultureInfo.InvariantCulture);

        return (T)constructor.Invoke(j.ToArray());

    }

}

var input = File.ReadLines("input.txt").Skip(2);
var regex = new Regex(@"/dev/grid/node-x(?<x>\d+)-y(?<y>\d+) +\d+T +(?<used>\d+)T +(?<avail>\d+)T +\d+%");

var nodes = (
    from line in input
    let data = RegexHelper.As<Data>(regex, line)
    select new Node(new Coordinate(data.x, data.y), data.used, data.avail)
    ).ToList();



// Filesystem              Size  Used  Avail  Use%
// /dev/grid/node-x0-y0     92T   68T    24T   73%

readonly record struct Data(int x, int y, int used, int avail);
readonly record struct Node(Coordinate c, int used, int avail);
readonly record struct Coordinate(int x, int y)
{
    public IEnumerable<Coordinate> Neighbours(int maxX, int maxY)
    {
        if (x > 0) yield return this with {x = x - 1};
        if (y > 0) yield return this with {y = y - 1};
        if (x < maxX) yield return this with {x = x + 1};
        if (y < maxY) yield return this with {y = y + 1};
    }
}


A viable pair is any two nodes (A,B), regardless of whether they are directly connected, such that:

* Node A is not empty (its Used is not zero).
* Nodes A and B are not the same node.
* The data on node A (its Used) would fit on node B (its Avail).

In [225]:

var viable = from a in nodes
             from b in nodes
             where a.c != b.c
             where a.used > 0
             where b.avail >= a.used
             select (a, b);



In [226]:

var table = (
    from node in nodes
    group node by node.c.y into g
    orderby g.Key
    let row = (from item in g
               orderby item.c.x
               select item.used switch 
               {
                    0 => "_",
                    > 100 => "#",
                    _ => "."
               }
               ).ToList()
    select row).ToList();

var empty = (from node in nodes where node.used == 0 select node.c).Single();
var walls = (from node in nodes where node.used > 100 select node.c).ToHashSet();
var goal = new Coordinate(36, 0);

foreach (var row in table.Take(10)) 
    Console.WriteLine(string.Join("", row));

int steps = 0;
for (int i = 0; i < 19; i++) empty = empty with {x = empty.x - 1};
steps += 19;
for (int i = 0; i < 6; i++) empty = empty with {y = empty.y - 1};
steps += 6;
for (int i = 0; i < 35; i++) empty = empty with {x = empty.x + 1};
steps += 35;
(goal, empty) = (empty, goal);
steps += 1;
for (int j = 0; j < 35; j++)
{
    for (int i = 0; i < 1; i++) empty = empty with {y = empty.y + 1};
    for (int i = 0; i < 2; i++) empty = empty with {x = empty.x - 1};
    for (int i = 0; i < 1; i++) empty = empty with {y = empty.y - 1};
    (goal, empty) = (empty, goal);
    steps += 5;
}
Console.WriteLine();
var sb = new StringBuilder();
for (int y = 0; y < 10; y++)
{
    for (int x = 0; x <37; x++)
    {
        var c = new Coordinate(x,y);
        if (c == empty)
            sb.Append('_');
        else if (c == goal)
            sb.Append('G');
        else if (walls.Contains(c))
            sb.Append('#');
        else
            sb.Append('.');
    }
    sb.AppendLine();
}
Console.WriteLine(sb.ToString());
steps




.....................................
.....................................
.....................................
.####################################
.....................................
.....................................
..................._.................
.....................................
.....................................
.....................................

G_...................................
.....................................
.....................................
.####################################
.....................................
.....................................
.....................................
.....................................
.....................................
.....................................



236